In [4]:
import pandas as pd
import scipy
import statsmodels

%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [5]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')

In [6]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [10]:
data.shape

(72, 15750)

In [12]:
data.Diagnosis.unique()

array(['normal', 'early neoplasia', 'cancer'], dtype=object)

In [16]:
data_normal = data.loc[data.Diagnosis == 'normal']
data_EN = data.loc[data.Diagnosis == 'early neoplasia']
data_cancer = data.loc[data.Diagnosis == 'cancer']

In [17]:
data_normal = data_normal.drop('Diagnosis', axis =1)
data_EN = data_EN.drop('Diagnosis', axis =1)
data_cancer = data_cancer.drop('Diagnosis', axis =1)

In [19]:
import scipy.stats as stats

In [27]:
scipy.stats.ttest_ind(data_normal['LOC643837'], data_EN['LOC643837'], equal_var = False)[1]

0.6907660157497355

In [28]:
list_pvalues = []

for name in data_normal.columns[1:]:
    p_value = scipy.stats.ttest_ind(data_normal[name], data_EN[name], equal_var = False)[1]
    list_pvalues.append(p_value)

In [29]:
list_pvalues_2 = []

for name in data_EN.columns[1:]:
    p_value = scipy.stats.ttest_ind(data_EN[name], data_cancer[name], equal_var = False)[1]
    list_pvalues_2.append(p_value)

In [37]:
indeces = [x[0] for x in enumerate(list_pvalues) if x[1] < 0.05]

In [35]:
indeces2 = [x[0] for x in enumerate(list_pvalues_2) if x[1] < 0.05]

In [61]:
with open('1_1.txt', 'w+') as file:
    file.write(str(len(indeces)))
file.close()

In [62]:
with open('1_2.txt', 'w+') as file:
    file.write(str(len(indeces2)))
file.close()

### Вторая часть

In [48]:
from statsmodels.sandbox.stats.multicomp import multipletests 

In [51]:
reject1, p_corrected_1, a1_1, a2_1 = multipletests(list_pvalues, 
                                            alpha = 0.025, 
                                            method = 'holm') 

In [52]:
reject2, p_corrected_2, a1_2, a2_2 = multipletests(list_pvalues_2, 
                                            alpha = 0.025, 
                                            method = 'holm') 

In [53]:
indeces1_holm = [x[0] for x in enumerate(p_corrected_1) if x[1] < 0.025]

In [54]:
indeces2_holm = [x[0] for x in enumerate(p_corrected_2) if x[1] < 0.025]

In [65]:
names_1 = pd.Series(data_normal.columns[1:]).loc[indeces1_holm]

In [67]:
names_2 = pd.Series(data_EN.columns[1:]).loc[indeces2_holm]

In [68]:
def fold_norm_EN(names):
    ret_names = []
    for name in names:
        C = data_normal[name].mean()
        T = data_EN[name].mean()
        if T > C:
            f = T/C
        else:
            f = -C/T
        
        if abs(f) > 1.5:
            ret_names.append(name)
    
    return ret_names

In [72]:
names_1_f = fold_norm_EN(names_1)

In [73]:
with open('2_1.txt', 'w+') as file:
    file.write(str(len(names_1_f)))
file.close()

In [88]:
def fold_EN_cancer(names):
    ret_names = []
    for name in names:
        C = data_EN[name].mean()
        T = data_cancer[name].mean()
        if T > C:
            f = T/C
        else:
            f = -C/T
        
        if abs(f) > 1.5:
            ret_names.append(name)
    
    return ret_names

In [89]:
names_2_f = fold_EN_cancer(names_2)

In [90]:
with open('2_2.txt', 'w+') as file:
    file.write(str(len(names_2_f)))
file.close()

### 3-я часть

In [77]:
reject1, p_corrected_1_bh, a1_1, a2_1 = multipletests(list_pvalues, 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 

In [78]:
reject2, p_corrected_2_bh, a1_2, a2_2 = multipletests(list_pvalues_2, 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 

In [79]:
indeces1_bh = [x[0] for x in enumerate(p_corrected_1_bh) if x[1] < 0.025]

In [80]:
indeces2_bh = [x[0] for x in enumerate(p_corrected_2_bh) if x[1] < 0.025]

In [81]:
names_1_bh = pd.Series(data_normal.columns[1:]).loc[indeces1_bh]

In [82]:
names_2_bh = pd.Series(data_EN.columns[1:]).loc[indeces2_bh]

In [84]:
names_1_f_bh = fold_norm_EN(names_1_bh)

In [85]:
with open('3_1.txt', 'w+') as file:
    file.write(str(len(names_1_f_bh)))
file.close()

In [91]:
names_2_f_bh = fold_EN_cancer(names_2_bh)

In [92]:
with open('3_2.txt', 'w+') as file:
    file.write(str(len(names_2_f_bh)))
file.close()